In [1]:
#Loading data from csv file in S3
data = sqlContext.read.csv('s3a://decisiontreespark/fraud_detection_data.csv', header=True,inferSchema=True)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1592028465843_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#View the columns
data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']

In [3]:
#Select required columns
sel_cols = ['step', 'type', 'amount', 
            'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
            'newbalanceDest', 'isFraud', 'isFlaggedFraud']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Create Dataframe from selected columns
df = data.select(sel_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#View the dataframe
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+
|step|    type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+
|   1| PAYMENT|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|
|   1| PAYMENT|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|
|   1|TRANSFER|  181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+
only showing top 3 rows

In [6]:
#View the "type" of transactions
df.select("type").show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|    type|
+--------+
| PAYMENT|
| PAYMENT|
|TRANSFER|
+--------+
only showing top 3 rows

In [7]:
#Group the transactions by "type"
df.groupby("type").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+
|    type| count|
+--------+------+
|CASH_OUT|362676|
| CASH_IN|218673|
| PAYMENT|329752|
|   DEBIT|  6474|
|TRANSFER| 82424|
+--------+------+

In [8]:
#Group by fraud or not 
df.groupby("isFraud").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|isFraud| count|
+-------+------+
|      1|   535|
|      0|999464|
+-------+------+

## String Indexer

In [9]:
#Import required libraries for string indexer
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#Apply function for string indexer
si = StringIndexer(inputCol="type", outputCol="type_index")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Apply required transformation on dataframe
df_indexed = si.fit(df).transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#View the dataframe
df_indexed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|       0.0|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|             0|       1.0|
+----+--------+--------+-------------+--------------+---

## One hot encoder 

In [13]:
#Import required libraries for one hot encoder
from pyspark.ml.feature import OneHotEncoderEstimator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Apply function for one hot encoder
encoder = OneHotEncoderEstimator(inputCols=["type_index"], 
                                 outputCols=["type_encoded"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Fit the indexed data
df_enc = encoder.fit(df_indexed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Transform based on indexed data
df_encoded = df_enc.transform(df_indexed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#view the dataframe
df_encoded.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|step|    type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|   1| PAYMENT|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1| PAYMENT|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1|TRANSFER|  181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
only showing top 3 rows

## Vector Assembler

In [18]:
#Import required libraries for vector assembler
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#View encoded columns
df_encoded.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud', 'type_index', 'type_encoded']

In [20]:
#Select required columns
selected_cols = ['step', 'amount', 'oldbalanceOrg', 
                 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',  
                 'type_encoded']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Apply function for one hot encoder
va = VectorAssembler(inputCols=selected_cols,outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#transform the dataframe
df_vectors = va.transform(df_encoded)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#View the dataframe
df_vectors.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|step|    type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|   1| PAYMENT|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1| PAYMENT|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1|TRANSFER|  181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|(10,[0,1,2,9],[1....|
|   1|CASH_OUT|  181.0|        181.0|           0.0|       21182.0|           0.0|      

## Create Train Test data

In [24]:
#Divide the data into test and train data
(trainingData, testData) = df_vectors.randomSplit([0.7,0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#View the count of training data
trainingData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

700001

In [26]:
#View the count of test data
testData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

299998

## Decision Tree Classifier

In [27]:
#import the libraries required for creating a Decision Tree
from pyspark.ml.classification import DecisionTreeClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#Run the decision tree classifier function
dt = DecisionTreeClassifier(featuresCol="features", labelCol="isFraud")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Fit the training data
model = dt.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#Tranform the test data
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#View the predcitions
predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|step|   type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|   rawPrediction|         probability|prediction|
+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|   1|CASH_IN| 783.31|   8150331.93|    8151115.24|       2013.12|       1229.81|      0|             0|       2.0|(4,[2],[1.0])|[1.0,783.31,81503...|[641970.0,155.0]|[0.99975861397702...|       0.0|
|   1|CASH_IN| 863.08|   9290756.54|    9291619.62|       5577.88|        4714.8|      0|             0|       2.0|(4,[2],[1.0])|[1.0,863.08,92907...|[641970.0,155.0]|[0.99975861397702...|       0.0|


In [32]:
#View predictions, isFraud and features corresponding to them
predictions.select("prediction", "isFraud", "features").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------------+
|prediction|isFraud|            features|
+----------+-------+--------------------+
|       0.0|      0|[1.0,783.31,81503...|
|       0.0|      0|[1.0,863.08,92907...|
|       0.0|      0|[1.0,911.76,13356...|
|       0.0|      0|[1.0,2643.45,6434...|
|       0.0|      0|[1.0,4865.48,7395...|
+----------+-------+--------------------+
only showing top 5 rows

## Accuracy

In [33]:
#import the libraries required for calculating accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#Calculate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="isFraud",
                                  predictionCol="prediction",
                                  metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#view accuracy
accuracy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997366649110994

In [36]:
#Check the number of fruad transactions
predictions.groupby("isFraud").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|isFraud| count|
+-------+------+
|      1|   150|
|      0|299848|
+-------+------+

In [37]:
#Check the predicted values
predictions.groupby("prediction").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+
|prediction| count|
+----------+------+
|       1.0|   113|
|       0.0|299885|
+----------+------+

In [38]:
#view the predictions
predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|step|   type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|   rawPrediction|         probability|prediction|
+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|   1|CASH_IN| 783.31|   8150331.93|    8151115.24|       2013.12|       1229.81|      0|             0|       2.0|(4,[2],[1.0])|[1.0,783.31,81503...|[641970.0,155.0]|[0.99975861397702...|       0.0|
|   1|CASH_IN| 863.08|   9290756.54|    9291619.62|       5577.88|        4714.8|      0|             0|       2.0|(4,[2],[1.0])|[1.0,863.08,92907...|[641970.0,155.0]|[0.99975861397702...|       0.0|


# Confusion matrix

In [39]:
#import the libraries required to print the confusion matrix
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

important: need to cast to values to float type

In [40]:
#Place the predictions and labels together
preds_and_labels = predictions.select(['prediction','isFraud']).withColumn('label', F.col('isFraud').cast(FloatType())).orderBy('prediction')

preds_and_labels = preds_and_labels.select(['prediction','label'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#convert into RDD
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#print in form of an array
print(metrics.confusionMatrix().toArray())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[2.99827e+05 2.10000e+01]
 [5.80000e+01 9.20000e+01]]

# Precision 

In [43]:
#Calculate precision
precision = metrics.precision()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#View precision
precision

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997366649110994

# Recall

In [45]:
#Calculate recall
recall = metrics.recall()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#View recall
recall

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997366649110994

# f1Score

In [47]:
#Calculate f1 score
f1Score = metrics.fMeasure()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
#View f1 score
f1Score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997366649110994